### Imports

In [1]:
# Manipulação de dados
import numpy as np
import re

import pandas as pd
pd.set_option('display.max_columns', None)


### Pré-processamento

In [2]:
# Amostra dos dados de arquivo
# df_perfil = pd.read_csv('perfil_eleitorado_2020.csv', encoding = 'latin1', sep=';', nrows = 10)

# Upload todos os dados
df_perfil = pd.read_csv('perfil_eleitorado_2020.csv', encoding = 'latin1', sep=';')


#### Criação de Tabelas para MER

In [3]:
# Excluindo todos as tuplas que não sejam de São Paulo
df_perfil.drop(df_perfil[df_perfil['SG_UF'] != 'SP'].index, inplace=True)

# Tabela de Municípios
df_municipios = df_perfil[['CD_MUNICIPIO', 'NM_MUNICIPIO']].copy()
df_municipios.drop_duplicates(inplace=True)
df_municipios.sort_values(by='CD_MUNICIPIO', inplace=True)
df_municipios

,CD_MUNICIPIO,NM_MUNICIPIO
3355494,61000,EUCLIDES DA CUNHA PAULISTA
2242500,61018,ADAMANTINA
2405473,61026,BOREBI
3660443,61034,ADOLFO
2463853,61042,TARUMÃ
...,...,...
1266455,72397,VIRADOURO
2945147,72419,VISTA ALEGRE DO ALTO
2955128,72435,VOTORANTIM
2666082,72451,VOTUPORANGA


In [4]:
# Tabela de Gênero
df_genero = df_perfil[['CD_GENERO', 'DS_GENERO']].copy()
df_genero.drop_duplicates(inplace=True)
df_genero.sort_values(by='CD_GENERO', inplace=True)
df_genero

,CD_GENERO,DS_GENERO
220,0,NÃO INFORMADO
235,2,MASCULINO
0,4,FEMININO


In [5]:
# Tabela para Faixa Etária
df_faixaEtaria = df_perfil[['CD_FAIXA_ETARIA', 'DS_FAIXA_ETARIA']].copy()
df_faixaEtaria.drop_duplicates(inplace=True)
df_faixaEtaria.sort_values(by='CD_FAIXA_ETARIA', inplace=True)
df_faixaEtaria

,CD_FAIXA_ETARIA,DS_FAIXA_ETARIA
7550,-3,Inválido
239,1600,16 anos
243,1700,17 anos
248,1800,18 anos
430,1900,19 anos
438,2000,20 anos
150,2124,21 a 24 anos
152,2529,25 a 29 anos
34,3034,30 a 34 anos
35,3539,35 a 39 anos


In [6]:
# Tabela para Grau de Escolaridade
df_grauEscolar = df_perfil[['CD_GRAU_ESCOLARIDADE', 'DS_GRAU_ESCOLARIDADE']].copy()
df_grauEscolar.drop_duplicates(inplace=True)
df_grauEscolar.sort_values(by='CD_GRAU_ESCOLARIDADE', inplace=True)
df_grauEscolar

,CD_GRAU_ESCOLARIDADE,DS_GRAU_ESCOLARIDADE
1253789,0,NÃO INFORMADO
5,1,ANALFABETO
6,2,LÊ E ESCREVE
0,3,ENSINO FUNDAMENTAL INCOMPLETO
1,4,ENSINO FUNDAMENTAL COMPLETO
2,5,ENSINO MÉDIO INCOMPLETO
3,6,ENSINO MÉDIO COMPLETO
17,7,SUPERIOR INCOMPLETO
4,8,SUPERIOR COMPLETO


#### Drop de colunas desnecessárias

In [7]:
df_perfil.drop(columns = ['DT_GERACAO', # 01/08/2020
                          'HH_GERACAO', # 08:37:48
                          'ANO_ELEICAO', # 2020
                          'NM_MUNICIPIO', # Coluna para MER
                          'DS_MUN_SIT_BIOMETRIA', # Coluna para MER
                          'DS_GENERO', # Coluna para MER
                          'DS_FAIXA_ETARIA', # Coluna para MER
                          'DS_GRAU_ESCOLARIDADE', # Coluna para MER
                          'DS_ESTADO_CIVIL', # Coluna para MER
                          'CD_MUN_SIT_BIOMETRIA', # Dígito '0'
                          'DS_MUN_SIT_BIOMETRIA', # Sem Biometria
                          'SG_UF', # Dados limpos
                          ] , inplace=True)

In [8]:
df_perfil

,CD_MUNICIPIO,NR_ZONA,CD_GENERO,CD_ESTADO_CIVIL,CD_FAIXA_ETARIA,CD_GRAU_ESCOLARIDADE,QT_ELEITORES_PERFIL,QT_ELEITORES_BIOMETRIA,QT_ELEITORES_DEFICIENCIA,QT_ELEITORES_INC_NM_SOCIAL
0,70734,314,4,3,7074,3,47,37,1,0
1,70734,314,4,3,7074,4,5,4,0,0
2,70734,314,4,3,7074,5,1,1,0,0
3,70734,314,4,3,7074,6,10,8,0,0
4,70734,314,4,3,7074,8,23,18,0,0
...,...,...,...,...,...,...,...,...,...,...
4248331,72451,147,2,3,5559,6,512,500,0,0
4248332,72451,147,2,3,5559,7,41,40,0,0
4248333,72451,147,2,3,5559,8,323,314,2,0
4248334,72451,147,2,3,6064,1,26,23,0,0


#### Exportação das Bases

In [9]:
# Exportação das bases para otimizar memória RAM e tabelas MER
df_perfil.to_excel('perfil_eleitorado_clean.xlsx', index=False)
df_municipios.to_excel('perfil_municipio.xlsx', index=False)
df_genero.to_excel('perfil_genero.xlsx', index=False)
df_faixaEtaria.to_excel('perfil_faixaEtaria.xlsx', index=False)
df_grauEscolar.to_excel('perfil_grauEscolar.xlsx', index=False)